In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
model = keras.models.load_model('models/defthomes_musevana')
data = open('data/corpus/defthomes_musevana_processed.txt').read()

In [ ]:
tokenizer = Tokenizer()

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
seed_texts = [
"I watched you change Into a fly",
"I looked away You were on fire",
"I watched a change in you It's like you never had wings",
"Now, you feel so alive I've watched you change",
"I'm a rabbit in your headlights Scared of the spotlight",
"You don't come to visit I'm stuck in this bed",
"Thin rubber gloves She laughs when she's cryin'",
"She cries when she's laughin'",
"My knife it's sharp and chrome Come see inside my bones",
"All of the fiends are on the block I'm the new king",
"I take the queen",
"When you were here before Couldn't look you in the eye",
"You're just like an angel Your skin makes me cry",
"You float like a feather In a beautiful world",
"I wish I was special You're so fucking special",
"But I'm a creep I'm a weirdo",
"What the hell am I doing here? I don't belong here",
"Standing in the middle of a hurricane",
"rooted in the ground things tearing apart so organized",
"These shadows don't need the light Spinning fast, screaming loud",
"touching you gently harmony of destruction",
"so quiet This noise don’t need air"
]


In [ ]:
next_words = 100

for i, seed_text in enumerate(seed_texts):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)
    outfile = 'data/generated/ ' + str(i) + str(next_words) + '.txt'
    with open(outfile, "w") as text_file:
        text_file.write(seed_text)

In [ ]:
# Rmoving seed texts
texts = glob.glob('data/generated/*.txt')

for text in texts:
    f = open(text,'r')
    lst = []
    for line in f:
        for word in seed_texts:
            if word in line:
                line = line.replace(word,'')
        lst.append(line)
    f.close()
    f = open(text,'w')
    for line in lst:
        f.write(line)
    f.close()

# Generating MIDIs
Get vowels and assing notes to each one.import collections

In [1]:
import glob
from utils.generator import *

In [2]:
vh = {
    'O' : 45,
    'B' : 60,
    'C' : 61,
    'D' : 62,
    'E' : 63,
    'F' : 64,
    'G' : 65,
    'A' : 66,
    'H' : 67,
    'I' : 68,
    'J' : 69,
    'K' : 70,
    'L' : 71,
    'M' : 72,
    'N' : 73,
    'P' : 74,
}

In [3]:
get_midi_from_vowels('data/generated/control-phoetics.txt', '1', vh)

In [ ]:
texts = glob.glob('data/generated/*.txt', encoder=vh)
for text in texts:
    get_midi_from_vowels(text, encoder)

In [ ]:
from itertools import permutations
texts = glob.glob('data/generated/*.txt')

letters = ['a', 'e', 'i', 'o', 'u']
notes = list(permutations([57, 60, 62, 64, 67]))

for i in range(len(notes)):
    encoder_permutation = {letters[x]: notes[i][x] for x in range(len(letters))}
    encoder_permutation['O'] = 45
    for text in texts:
        get_midi_from_vowels(text, postfix=str(i), encoder=encoder_permutation)
    